In [ ]:
#问题二附件3语音业务打分预测代码
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')
plt.rcParams['font.sans-serif'] = ['SimHei']
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.model_selection import cross_val_score
from sklearn.metrics import accuracy_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from lightgbm import LGBMClassifier
from sklearn.tree import DecisionTreeClassifier
from xgboost import XGBClassifier
from sklearn.multiclass import OneVsRestClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.multiclass import OneVsOneClassifier
from mlxtend.classifier import StackingClassifier

In [ ]:
data_3 = pd.read_excel('./data/data_1处理后的数据.xlsx').drop('Unnamed: 0', axis = 1)

In [ ]:
# 多标签多分类问题
df1 = data_3.copy()
df1 = pd.get_dummies(df1)

In [ ]:
# 基模型选择随机森林时
from sklearn.multioutput import ClassifierChain
from sklearn.metrics import jaccard_score
X_train, X_test, y_train, y_test = train_test_split(df1.iloc[:, 4 : ], df1.iloc[:, :4], test_size = 0.3, random_state = 10)
model_1 = RandomForestClassifier()
chains = [ClassifierChain(model_1, order='random', random_state=i)
          for i in range(10)]
for chain in chains:
    chain.fit(X_train, y_train)
Y_pred_chains = np.array([chain.predict(X_test) for chain in chains], dtype = 'int64')

jac_scores = []
for i in range(len(chains)):
    for j in range(len(y_test.to_numpy())):
        a = list(y_test.to_numpy()[j])
        b = list(Y_pred_chains[i][j])
        jac_score = jaccard_score(a, b, average = None)
        jac_scores.append(jac_score[0])
    sums = 0
    for n in jac_scores:
        sums += n
    avg = sums / len(jac_scores)
    print('第%i条链的平均Jaccard相似系数为:%.3f'%((i + 1), avg))

In [ ]:
# 基模型选择LightGBM时
model_3 = LGBMClassifier()
chains = [ClassifierChain(model_3, order='random', random_state=i)
          for i in range(10)]
for chain in chains:
    chain.fit(X_train, y_train)
Y_pred_chains = np.array([chain.predict(X_test) for chain in chains], dtype = 'int64')


jac_scores = []
for i in range(len(chains)):
    for j in range(len(y_test.to_numpy())):
        a = list(y_test.to_numpy()[j])
        b = list(Y_pred_chains[i][j])
        jac_score = jaccard_score(a, b, average = None)
        jac_scores.append(jac_score[0])
    sums = 0
    for n in jac_scores:
        sums += n
    avg = sums / len(jac_scores)
    print('第%i条链的平均Jaccard相似系数为:%.3f'%((i + 1), avg))

    
for i in range(len(chains)):
    y_pred = pd.DataFrame(Y_pred_chains[i])
    scores = []
    for j in range(4):
        score = accuracy_score(y_test.iloc[:, j], y_pred.iloc[:, j])
        scores.append(score)
    print('第%i条链的准确率为:'%((i + 1)))
    print(scores)

In [ ]:
# 语音整理满意度
"""
不采用过采样
"""
## 采用100%的特征
cols = ['语音通话整体满意度', '是否遇到过网络问题', 'GPRS总流量（KB）', '当月ARPU', '前3月MOU', '当月MOU', '语音通话-时长（分钟）', 'mos质差次数', \
    '终端品牌', '脱网次数', '通话中有杂音、听不清、断断续续', 'GPRS-国内漫游-流量（KB）', '未接通掉话次数', '有信号无法拨通', '客户星级标识', \
        '办公室', '前第3个月欠费金额', '通话过程中突然中断', '套外流量费（元）', '手机没有信号', '省际漫游-时长（分钟）', '套外流量（MB）', \
            '当月欠费金额', '居民小区', '通话过程中一方听不见', '地铁', '高校', '高铁', '其他，请注明.1', '外省语音占比', '外省流量占比', \
                '商业街', '农村', '串线', '其他，请注明']
df_3 = pd.DataFrame()
for i in cols:
    df_3[i] = data_3[i]


df_3 = pd.get_dummies(df_3)

In [ ]:
## 尝试采用投票法
from sklearn.ensemble import VotingClassifier
X_train, X_test, y_train, y_test = train_test_split(df_3.iloc[:, 1 : ], df_3.iloc[:, 0], test_size = 0.3, shuffle = True, random_state = 10)
clf1 = XGBClassifier(n_estimators =300, random_state=10)
clf2 = RandomForestClassifier(n_estimators = 300, max_depth = 5, random_state = 10)
clf3 = LGBMClassifier(max_depth = 5, n_estimators = 300, random_state = 10)
clf4 = GradientBoostingClassifier(n_estimators = 300, random_state = 10, max_depth = 5)
eclf = VotingClassifier(estimators=[('xgb', clf1), ('rf', clf2), ('lgb', clf3), ('gbdt', clf4)], voting = 'hard')
y_test = y_test - 1
for clf, label in zip([clf1, clf2, clf3, clf4, eclf], ['XGBBoosting', 'Random Forest', 'LightGBM', 'GBDT', 'Ensemble']):
    scores = cross_val_score(clf, X_train, y_train - 1, cv = 5, scoring='accuracy')
    print("Accuracy: %0.2f (+/- %0.2f) [%s]" % (scores.mean(), scores.std(), label))

In [ ]:
## 尝试模型融合
from mlxtend.classifier import StackingClassifier
sclf = StackingClassifier(classifiers = [clf1, clf3], meta_classifier = clf2)
for clf, label in zip([clf2, clf3, sclf], ['Random Forest', 'LightGBM', 'Stacking Model']):
    scores = cross_val_score(clf, X_train, y_train - 1, cv = 5, scoring = 'accuracy')
    print("Accuracy: %0.2f (+/- %0.2f) [%s]" % (scores.mean(), scores.std(), label))

In [ ]:
## 尝试对随机森林进行调参
X_train, X_test, y_train, y_test = train_test_split(df_3.iloc[:, 1 : ], df_3.iloc[:, 0], test_size = 0.3, shuffle = True, random_state = 10)
clf2.fit(X_train, y_train)
y_pred = clf2.predict(X_test)
print(classification_report(y_test, y_pred))
print(cross_val_score(clf2, X_train, y_train, cv = 5, scoring = 'accuracy'))

In [ ]:
from sklearn.model_selection import GridSearchCV
X_train, X_test, y_train, y_test = train_test_split(df_3.iloc[:, 1 : ], df_3.iloc[:, 0], test_size = 0.3, shuffle = True, random_state = 10)
param = {"criterion": ["gini", "entropy"], "max_depth": np.arange(1, 20, 1), 'n_estimators': np.arange(100, 501, 50)} 
rf_gs = GridSearchCV(estimator = RandomForestClassifier(), param_grid = param, cv = 5, scoring = "accuracy", n_jobs = -1, verbose = 10) 
rf_gs.fit(X_train, y_train) 
print(rf_gs.best_params_) 
y_hat = rf_gs.best_estimator_.predict(X_test) 
print(classification_report(y_test, y_hat))

In [ ]:
## OVR算法
plt.rcParams['font.sans-serif'] = ['SimHei']
X_train, X_test, y_train, y_test = train_test_split(df_3.iloc[:, 1 : ], df_3.iloc[:, 0], test_size = 0.3, shuffle = True, random_state = 10)
m1 = RandomForestClassifier(n_estimators = 300 , max_depth = 5, random_state = 10)
m2 = DecisionTreeClassifier(random_state = 10, max_depth = 5)
m3 = GradientBoostingClassifier(random_state = 10, n_estimators = 300, max_depth = 5)
m4 = LGBMClassifier(random_state = 10, n_estimators = 300, max_depth = 5)
m5 = XGBClassifier(random_state = 10, n_estimators = 300)
models =[m1, m2, m3, m4, m5]
scores = []
for model in models:
    clf_multilabel = OneVsRestClassifier(model)
    clf_multilabel.fit(X_train, y_train)
    y_pred = clf_multilabel.predict(X_test)
    score = accuracy_score(y_test, y_pred)
    print('分类报告为:%s'%(model))
    print(classification_report(y_test, y_pred))
    print('%s五折交叉验证的分数为:'%(model))
    print(cross_val_score(clf_multilabel, X_train, y_train, cv = 5))
    scores.append(score)
model_names = ('Random Forest',
               'Decision Tree',
               'GBDT',
               'LightGBM',
               'Xgboost')
x_pos = np.arange(len(model_names))
fig, ax = plt.subplots(figsize = (12, 8), dpi = 512)
ax.grid(True)
ax.set_title('One Vs Rest Classifier', font = {'family' : 'STSong', 'size' : 14})
ax.set_xticks(x_pos)
ax.set_xticklabels(model_names, font = {'family' : 'STSong', 'size' : 14})
ax.set_ylabel('model test scores', font = {'family' : 'STSong', 'size' : 14})
ax.set_ylim([min(scores) * .9, max(scores) * 1.05])
colors = ['r'] + ['b'] * (len(scores))
ax.bar(x_pos, scores, alpha = 0.5, color = colors)
plt.tight_layout()
plt.show()

In [ ]:
## OVR算法的随机森林调参
param = {"estimator__max_depth": np.arange(1,20,1), 'estimator__n_estimators': np.arange(100, 501, 50)}
model = OneVsRestClassifier(RandomForestClassifier(random_state = 10))
clf = GridSearchCV(model, param_grid = param, scoring = 'accuracy', n_jobs = -1, cv = 5, verbose = 2)
clf.fit(X_train, y_train)
print(clf.best_params_)
y_hat = clf.best_estimator_.predict(X_test)
print(classification_report(y_test, y_hat))

In [ ]:
## OVO算法
from sklearn.multiclass import OneVsOneClassifier
plt.rcParams['font.sans-serif'] = ['SimHei']
X_train, X_test, y_train, y_test = train_test_split(df_3.iloc[:, 1 : ], df_3.iloc[:, 0], test_size = 0.3, shuffle = True, random_state = 10)
m1 = RandomForestClassifier(n_estimators = 300 , max_depth = 5, random_state = 10)
m2 = DecisionTreeClassifier(random_state = 10, max_depth = 5)
m3 = GradientBoostingClassifier(random_state = 10, n_estimators = 300, max_depth = 5)
m4 = LGBMClassifier(random_state = 10, n_estimators = 300, max_depth = 5)
m5 = XGBClassifier(random_state = 10, n_estimators = 300)
models =[m1, m2, m3, m4, m5]
scores = []
for model in models:
    clf_multilabel = OneVsOneClassifier(model)
    clf_multilabel.fit(X_train, y_train)
    y_pred = clf_multilabel.predict(X_test)
    score = accuracy_score(y_test, y_pred)
    print('分类报告为:%s'%(model))
    print(classification_report(y_test, y_pred))
    print('%s五折交叉验证的分数为:'%(model))
    print(cross_val_score(clf_multilabel, X_train, y_train, cv = 5))
    scores.append(score)
model_names = ('Random Forest',
               'Decision Tree',
               'GBDT',
               'LightGBM',
               'Xgboost')
x_pos = np.arange(len(model_names))
fig, ax = plt.subplots(figsize = (12, 8), dpi = 512)
ax.grid(True)
ax.set_title('One Vs One Classifier', font = {'family' : 'STSong', 'size' : 14})
ax.set_xticks(x_pos)
ax.set_xticklabels(model_names, font = {'family' : 'STSong', 'size' : 14})
ax.set_ylabel('model test scores', font = {'family' : 'STSong', 'size' : 14})
ax.set_ylim([min(scores) * .9, max(scores) * 1.05])
colors = ['r'] + ['b'] * (len(scores))
ax.bar(x_pos, scores, alpha = 0.5, color = colors)
plt.tight_layout()
plt.show()

In [ ]:
## OVO算法对随机森林进行调参
param = {"estimator__max_depth": np.arange(1,20,1), 'estimator__n_estimators': np.arange(100, 501, 50)}
model = OneVsOneClassifier(RandomForestClassifier(random_state = 10))
clf = GridSearchCV(model, param_grid = param, scoring = 'accuracy', n_jobs = -1, cv = 5, verbose = 2)
clf.fit(X_train, y_train)
print(clf.best_params_)
y_hat = clf.best_estimator_.predict(X_test)
print(classification_report(y_test, y_hat))

In [ ]:
# 特征选取前80%
cols = ['语音通话整体满意度', '是否遇到过网络问题', 'GPRS总流量（KB）', '当月ARPU', '前3月MOU', '当月MOU', '语音通话-时长（分钟）', 'mos质差次数', \
    '终端品牌', '脱网次数', '通话中有杂音、听不清、断断续续', 'GPRS-国内漫游-流量（KB）', '未接通掉话次数', '有信号无法拨通', '客户星级标识', \
        '办公室', '前第3个月欠费金额', '通话过程中突然中断', '套外流量费（元）', '手机没有信号', '省际漫游-时长（分钟）', '套外流量（MB）', \
            '当月欠费金额', '居民小区', '通话过程中一方听不见', '地铁', '高校', '高铁']
df_4 = pd.DataFrame()
for i in cols:
    df_4[i] = data_3[i]


df_4 = pd.get_dummies(df_4)

In [ ]:
## 投票法
X_train, X_test, y_train, y_test = train_test_split(df_4.iloc[:, 1 : ], df_4.iloc[:, 0], test_size = 0.3, shuffle = True, random_state = 10)
clf1 = XGBClassifier(n_estimators =300, random_state=10)
clf2 = RandomForestClassifier(n_estimators = 300, max_depth = 5, random_state = 10)
clf3 = LGBMClassifier(max_depth = 5, n_estimators = 300, random_state = 10)
clf4 = GradientBoostingClassifier(n_estimators = 300, random_state = 10, max_depth = 5)
eclf = VotingClassifier(estimators=[('xgb', clf1), ('rf', clf2), ('lgb', clf3), ('gbdt', clf4)], voting = 'hard')
y_test = y_test - 1
for clf, label in zip([clf1, clf2, clf3, clf4, eclf], ['XGBBoosting', 'Random Forest', 'LightGBM', 'GBDT', 'Ensemble']):
    scores = cross_val_score(clf, X_train, y_train - 1, cv = 5, scoring='accuracy')
    print("Accuracy: %0.2f (+/- %0.2f) [%s]" % (scores.mean(), scores.std(), label))

In [ ]:
# 模型融合
sclf = StackingClassifier(classifiers = [clf1, clf3], meta_classifier = clf2)
for clf, label in zip([clf1, clf3, sclf], ['XGboost', 'LightGBM', 'Stacking Model']):
    scores = cross_val_score(clf, X_train, y_train - 1, cv = 5, scoring = 'accuracy')
    print("Accuracy: %0.2f (+/- %0.2f) [%s]" % (scores.mean(), scores.std(), label))


sclf = StackingClassifier(classifiers = [clf2, clf3], meta_classifier = clf1)
for clf, label in zip([clf2, clf3, sclf], ['Random Forest', 'LightGBM', 'Stacking Model']):
    scores = cross_val_score(clf, X_train, y_train - 1, cv = 5, scoring = 'accuracy')
    print("Accuracy: %0.2f (+/- %0.2f) [%s]" % (scores.mean(), scores.std(), label))


sclf = StackingClassifier(classifiers = [clf1, clf2], meta_classifier = clf3)
for clf, label in zip([clf1, clf2, sclf], ['XGboost', 'Random Forest', 'Stacking Model']):
    scores = cross_val_score(clf, X_train, y_train - 1, cv = 5, scoring = 'accuracy')
    print("Accuracy: %0.2f (+/- %0.2f) [%s]" % (scores.mean(), scores.std(), label))

In [ ]:
## 对随机森林进行调参
X_train, X_test, y_train, y_test = train_test_split(df_4.iloc[:, 1 : ], df_4.iloc[:, 0], test_size = 0.3, shuffle = True, random_state = 10)
param = {"criterion": ["gini", "entropy"], "max_depth": np.arange(1, 20, 1), 'n_estimators': np.arange(100, 501, 50)}
rf_gs = GridSearchCV(estimator = RandomForestClassifier(), param_grid = param, cv = 5, scoring = "accuracy", n_jobs = -1, verbose = 10)
rf_gs.fit(X_train, y_train)
print(rf_gs.best_params_)
y_hat = rf_gs.best_estimator_.predict(X_test)
print(classification_report(y_test, y_hat))

In [ ]:
## OVR算法
plt.rcParams['font.sans-serif'] = ['SimHei']
X_train, X_test, y_train, y_test = train_test_split(df_4.iloc[:, 1 : ], df_4.iloc[:, 0], test_size = 0.3, shuffle = True, random_state = 10)
m1 = RandomForestClassifier(n_estimators = 300 , max_depth = 5, random_state = 10)
m2 = DecisionTreeClassifier(random_state = 10, max_depth = 5)
m3 = GradientBoostingClassifier(random_state = 10, n_estimators = 300, max_depth = 5)
m4 = LGBMClassifier(random_state = 10, n_estimators = 300, max_depth = 5)
m5 = XGBClassifier(random_state = 10, n_estimators = 300)
models =[m1, m2, m3, m4, m5]
scores = []
for model in models:
    clf_multilabel = OneVsRestClassifier(model)
    clf_multilabel.fit(X_train, y_train)
    y_pred = clf_multilabel.predict(X_test)
    score = accuracy_score(y_test, y_pred)
    print('分类报告为:%s'%(model))
    print(classification_report(y_test, y_pred))
    print('%s五折交叉验证的分数为:'%(model))
    print(cross_val_score(clf_multilabel, X_train, y_train, cv = 5))
    scores.append(score)
model_names = ('Random Forest',
               'Decision Tree',
               'GBDT',
               'LightGBM',
               'Xgboost')
x_pos = np.arange(len(model_names))
fig, ax = plt.subplots(figsize = (12, 8), dpi = 512)
ax.grid(True)
ax.set_title('One Vs Rest Classifier', font = {'family' : 'STSong', 'size' : 14})
ax.set_xticks(x_pos)
ax.set_xticklabels(model_names, font = {'family' : 'STSong', 'size' : 14})
ax.set_ylabel('model test scores', font = {'family' : 'STSong', 'size' : 14})
ax.set_ylim([min(scores) * .9, max(scores) * 1.05])
colors = ['r'] + ['b'] * (len(scores))
ax.bar(x_pos, scores, alpha = 0.5, color = colors)
plt.tight_layout()
plt.show()

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(df_4.iloc[:, 1 : ], df_4.iloc[:, 0], test_size = 0.3, shuffle = True, random_state = 10)
param = {"estimator__max_depth": np.arange(1,20,1), 'estimator__n_estimators': np.arange(100, 501, 50)}
model = OneVsRestClassifier(RandomForestClassifier(random_state = 10))
clf = GridSearchCV(model, param_grid = param, scoring = 'accuracy', n_jobs = -1, cv = 5, verbose = 2)
clf.fit(X_train, y_train)
print(clf.best_params_)
y_hat = clf.best_estimator_.predict(X_test)
print(classification_report(y_test, y_hat))

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(df_4.iloc[:, 1 : ], df_4.iloc[:, 0], test_size = 0.3, shuffle = True, random_state = 10)
param = {"estimator__max_depth": np.arange(1,20,1), 'estimator__n_estimators': np.arange(100, 501, 50)}
model = OneVsRestClassifier(RandomForestClassifier(random_state = 10))
clf = GridSearchCV(model, param_grid = param, scoring = 'accuracy', n_jobs = -1, cv = 5, verbose = 2)
clf.fit(X_train, y_train)
print(clf.best_params_)
y_hat = clf.best_estimator_.predict(X_test)
print(classification_report(y_test, y_hat))

In [ ]:
## OVO算法
plt.rcParams['font.sans-serif'] = ['SimHei']
X_train, X_test, y_train, y_test = train_test_split(df_4.iloc[:, 1 : ], df_4.iloc[:, 0], test_size = 0.3, shuffle = True, random_state = 10)
m1 = RandomForestClassifier(n_estimators = 300 , max_depth = 5, random_state = 10)
m2 = DecisionTreeClassifier(random_state = 10, max_depth = 5)
m3 = GradientBoostingClassifier(random_state = 10, n_estimators = 300, max_depth = 5)
m4 = LGBMClassifier(random_state = 10, n_estimators = 300, max_depth = 5)
m5 = XGBClassifier(random_state = 10, n_estimators = 300)
models =[m1, m2, m3, m4, m5]
scores = []
for model in models:
    clf_multilabel = OneVsOneClassifier(model)
    clf_multilabel.fit(X_train, y_train)
    y_pred = clf_multilabel.predict(X_test)
    score = accuracy_score(y_test, y_pred)
    print('分类报告为:%s'%(model))
    print(classification_report(y_test, y_pred))
    print('%s五折交叉验证的分数为:'%(model))
    print(cross_val_score(clf_multilabel, X_train, y_train, cv = 5))
    scores.append(score)
model_names = ('Random Forest',
               'Decision Tree',
               'GBDT',
               'LightGBM',
               'Xgboost')
x_pos = np.arange(len(model_names))
fig, ax = plt.subplots(figsize = (12, 8), dpi = 512)
ax.grid(True)
ax.set_title('One Vs One Classifier', font = {'family' : 'STSong', 'size' : 14})
ax.set_xticks(x_pos)
ax.set_xticklabels(model_names, font = {'family' : 'STSong', 'size' : 14})
ax.set_ylabel('model test scores', font = {'family' : 'STSong', 'size' : 14})
ax.set_ylim([min(scores) * .9, max(scores) * 1.05])
colors = ['r'] + ['b'] * (len(scores))
ax.bar(x_pos, scores, alpha = 0.5, color = colors)
plt.tight_layout()
plt.show()

In [ ]:
param = {"estimator__max_depth": np.arange(1,20,1), 'estimator__n_estimators': np.arange(100, 501, 50)}
model = OneVsOneClassifier(RandomForestClassifier(random_state = 10))
clf = GridSearchCV(model, param_grid = param, scoring = 'accuracy', n_jobs = -1, cv = 5, verbose = 2)
clf.fit(X_train, y_train)
print(clf.best_params_)
y_hat = clf.best_estimator_.predict(X_test)
print(classification_report(y_test, y_hat))

In [ ]:
# 选取前50%的特征
cols = ['语音通话整体满意度', '是否遇到过网络问题', 'GPRS总流量（KB）', '当月ARPU', '前3月MOU', '当月MOU', '语音通话-时长（分钟）', 'mos质差次数', \
    '终端品牌', '脱网次数', '通话中有杂音、听不清、断断续续', 'GPRS-国内漫游-流量（KB）', '未接通掉话次数', '有信号无法拨通', '客户星级标识', \
        '办公室', '前第3个月欠费金额', '通话过程中突然中断']
df_5 = pd.DataFrame()
for i in cols:
    df_5[i] = data_3[i]

In [ ]:
df_5 = pd.get_dummies(df_5)

In [ ]:
## 投票法
X_train, X_test, y_train, y_test = train_test_split(df_5.iloc[:, 1 : ], df_5.iloc[:, 0], test_size = 0.3, shuffle = True, random_state = 10)
clf1 = XGBClassifier(n_estimators =300, random_state=10)
clf2 = RandomForestClassifier(n_estimators = 300, max_depth = 5, random_state = 10)
clf3 = LGBMClassifier(max_depth = 5, n_estimators = 300, random_state = 10)
clf4 = GradientBoostingClassifier(n_estimators = 300, random_state = 10, max_depth = 5)
eclf = VotingClassifier(estimators=[('xgb', clf1), ('rf', clf2), ('lgb', clf3), ('gbdt', clf4)], voting = 'hard')
y_test = y_test - 1
for clf, label in zip([clf1, clf2, clf3, clf4, eclf], ['XGBBoosting', 'Random Forest', 'LightGBM', 'GBDT', 'Ensemble']):
    scores = cross_val_score(clf, X_train, y_train - 1, cv = 5, scoring='accuracy')
    print("Accuracy: %0.2f (+/- %0.2f) [%s]" % (scores.mean(), scores.std(), label))

In [ ]:
## 模型融合
sclf = StackingClassifier(classifiers = [clf1, clf3], meta_classifier = clf2)
for clf, label in zip([clf1, clf3, sclf], ['XGboost', 'LightGBM', 'Stacking Model']):
    scores = cross_val_score(clf, X_train, y_train - 1, cv = 5, scoring = 'accuracy')
    print("Accuracy: %0.2f (+/- %0.2f) [%s]" % (scores.mean(), scores.std(), label))


sclf = StackingClassifier(classifiers = [clf2, clf3], meta_classifier = clf1)
for clf, label in zip([clf2, clf3, sclf], ['Random Forest', 'LightGBM', 'Stacking Model']):
    scores = cross_val_score(clf, X_train, y_train - 1, cv = 5, scoring = 'accuracy')
    print("Accuracy: %0.2f (+/- %0.2f) [%s]" % (scores.mean(), scores.std(), label))


sclf = StackingClassifier(classifiers = [clf1, clf2], meta_classifier = clf3)
for clf, label in zip([clf1, clf2, sclf], ['XGboost', 'Random Forest', 'Stacking Model']):
    scores = cross_val_score(clf, X_train, y_train - 1, cv = 5, scoring = 'accuracy')
    print("Accuracy: %0.2f (+/- %0.2f) [%s]" % (scores.mean(), scores.std(), label))

In [ ]:
## 调参
X_train, X_test, y_train, y_test = train_test_split(df_5.iloc[:, 1 : ], df_5.iloc[:, 0], test_size = 0.3, shuffle = True, random_state = 10)
param = {"criterion": ["gini", "entropy"], "max_depth": np.arange(1, 20, 1), 'n_estimators': np.arange(100, 501, 50)}
rf_gs = GridSearchCV(estimator = RandomForestClassifier(), param_grid = param, cv = 5, scoring = "accuracy", n_jobs = -1, verbose = 10)
rf_gs.fit(X_train, y_train)
print(rf_gs.best_params_)
y_hat = rf_gs.best_estimator_.predict(X_test)
print(classification_report(y_test, y_hat))

In [ ]:
## OVR算法
plt.rcParams['font.sans-serif'] = ['SimHei']
X_train, X_test, y_train, y_test = train_test_split(df_5.iloc[:, 1 : ], df_5.iloc[:, 0], test_size = 0.3, shuffle = True, random_state = 10)
m1 = RandomForestClassifier(n_estimators = 300 , max_depth = 5, random_state = 10)
m2 = DecisionTreeClassifier(random_state = 10, max_depth = 5)
m3 = GradientBoostingClassifier(random_state = 10, n_estimators = 300, max_depth = 5)
m4 = LGBMClassifier(random_state = 10, n_estimators = 300, max_depth = 5)
m5 = XGBClassifier(random_state = 10, n_estimators = 300)
models =[m1, m2, m3, m4, m5]
scores = []
for model in models:
    clf_multilabel = OneVsRestClassifier(model)
    clf_multilabel.fit(X_train, y_train)
    y_pred = clf_multilabel.predict(X_test)
    score = accuracy_score(y_test, y_pred)
    print('分类报告为:%s'%(model))
    print(classification_report(y_test, y_pred))
    print('%s五折交叉验证的分数为:'%(model))
    print(cross_val_score(clf_multilabel, X_train, y_train, cv = 5))
    scores.append(score)
model_names = ('Random Forest',
               'Decision Tree',
               'GBDT',
               'LightGBM',
               'Xgboost')
x_pos = np.arange(len(model_names))
fig, ax = plt.subplots(figsize = (12, 8), dpi = 512)
ax.grid(True)
ax.set_title('One Vs Rest Classifier', font = {'family' : 'STSong', 'size' : 14})
ax.set_xticks(x_pos)
ax.set_xticklabels(model_names, font = {'family' : 'STSong', 'size' : 14})
ax.set_ylabel('model test scores', font = {'family' : 'STSong', 'size' : 14})
ax.set_ylim([min(scores) * .9, max(scores) * 1.05])
colors = ['r'] + ['b'] * (len(scores))
ax.bar(x_pos, scores, alpha = 0.5, color = colors)
plt.tight_layout()
plt.show()

In [ ]:
param = {"estimator__max_depth": np.arange(1,20,1), 'estimator__n_estimators': np.arange(100, 501, 50)}
model = OneVsRestClassifier(RandomForestClassifier(random_state = 10))
clf = GridSearchCV(model, param_grid = param, scoring = 'accuracy', n_jobs = -1, cv = 5, verbose = 2)
clf.fit(X_train, y_train)
print(clf.best_params_)
y_hat = clf.best_estimator_.predict(X_test)
print(classification_report(y_test, y_hat))

In [ ]:
## OVO算法
plt.rcParams['font.sans-serif'] = ['SimHei']
X_train, X_test, y_train, y_test = train_test_split(df_4.iloc[:, 1 : ], df_4.iloc[:, 0], test_size = 0.3, shuffle = True, random_state = 10)
m1 = RandomForestClassifier(n_estimators = 300 , max_depth = 5, random_state = 10)
m2 = DecisionTreeClassifier(random_state = 10, max_depth = 5)
m3 = GradientBoostingClassifier(random_state = 10, n_estimators = 300, max_depth = 5)
m4 = LGBMClassifier(random_state = 10, n_estimators = 300, max_depth = 5)
m5 = XGBClassifier(random_state = 10, n_estimators = 300)
models =[m1, m2, m3, m4, m5]
scores = []
for model in models:
    clf_multilabel = OneVsOneClassifier(model)
    clf_multilabel.fit(X_train, y_train)
    y_pred = clf_multilabel.predict(X_test)
    score = accuracy_score(y_test, y_pred)
    print('分类报告为:%s'%(model))
    print(classification_report(y_test, y_pred))
    print('%s五折交叉验证的分数为:'%(model))
    print(cross_val_score(clf_multilabel, X_train, y_train, cv = 5))
    scores.append(score)
model_names = ('Random Forest',
               'Decision Tree',
               'GBDT',
               'LightGBM',
               'Xgboost')
x_pos = np.arange(len(model_names))
fig, ax = plt.subplots(figsize = (12, 8), dpi = 512)
ax.grid(True)
ax.set_title('One Vs One Classifier', font = {'family' : 'STSong', 'size' : 14})
ax.set_xticks(x_pos)
ax.set_xticklabels(model_names, font = {'family' : 'STSong', 'size' : 14})
ax.set_ylabel('model test scores', font = {'family' : 'STSong', 'size' : 14})
ax.set_ylim([min(scores) * .9, max(scores) * 1.05])
colors = ['r'] + ['b'] * (len(scores))
ax.bar(x_pos, scores, alpha = 0.5, color = colors)
plt.tight_layout()
plt.show()

In [ ]:
param = {"estimator__max_depth": np.arange(1,20,1), 'estimator__n_estimators': np.arange(100, 501, 50)}
model = OneVsOneClassifier(RandomForestClassifier(random_state = 10))
clf = GridSearchCV(model, param_grid = param, scoring = 'accuracy', n_jobs = -1, cv = 5, verbose = 2)
clf.fit(X_train, y_train)
print(clf.best_params_)
y_hat = clf.best_estimator_.predict(X_test)
print(classification_report(y_test, y_hat))

In [ ]:
# 提取前20%的特征
cols = ['语音通话整体满意度', '是否遇到过网络问题', 'GPRS总流量（KB）', '当月ARPU', '前3月MOU', '当月MOU', '语音通话-时长（分钟）', 'mos质差次数']
df_6 = pd.DataFrame()
for i in cols:
    df_6[i] = data_3[i]


df_6 = pd.get_dummies(df_6)

In [ ]:
## 投票法
X_train, X_test, y_train, y_test = train_test_split(df_6.iloc[:, 1 : ], df_6.iloc[:, 0], test_size = 0.3, shuffle = True, random_state = 10)
clf1 = XGBClassifier(n_estimators =300, random_state=10)
clf2 = RandomForestClassifier(n_estimators = 300, max_depth = 5, random_state = 10)
clf3 = LGBMClassifier(max_depth = 5, n_estimators = 300, random_state = 10)
clf4 = GradientBoostingClassifier(n_estimators = 300, random_state = 10, max_depth = 5)
eclf = VotingClassifier(estimators=[('xgb', clf1), ('rf', clf2), ('lgb', clf3), ('gbdt', clf4)], voting = 'hard')
y_test = y_test - 1
for clf, label in zip([clf1, clf2, clf3, clf4, eclf], ['XGBBoosting', 'Random Forest', 'LightGBM', 'GBDT', 'Ensemble']):
    scores = cross_val_score(clf, X_train, y_train - 1, cv = 5, scoring='accuracy')
    print("Accuracy: %0.2f (+/- %0.2f) [%s]" % (scores.mean(), scores.std(), label))


In [ ]:
## 模型融合
sclf = StackingClassifier(classifiers = [clf1, clf3], meta_classifier = clf2)
for clf, label in zip([clf1, clf3, sclf], ['XGboost', 'LightGBM', 'Stacking Model']):
    scores = cross_val_score(clf, X_train, y_train - 1, cv = 5, scoring = 'accuracy')
    print("Accuracy: %0.2f (+/- %0.2f) [%s]" % (scores.mean(), scores.std(), label))


sclf = StackingClassifier(classifiers = [clf2, clf3], meta_classifier = clf1)
for clf, label in zip([clf2, clf3, sclf], ['Random Forest', 'LightGBM', 'Stacking Model']):
    scores = cross_val_score(clf, X_train, y_train - 1, cv = 5, scoring = 'accuracy')
    print("Accuracy: %0.2f (+/- %0.2f) [%s]" % (scores.mean(), scores.std(), label))


sclf = StackingClassifier(classifiers = [clf1, clf2], meta_classifier = clf3)
for clf, label in zip([clf1, clf2, sclf], ['XGboost', 'Random Forest', 'Stacking Model']):
    scores = cross_val_score(clf, X_train, y_train - 1, cv = 5, scoring = 'accuracy')
    print("Accuracy: %0.2f (+/- %0.2f) [%s]" % (scores.mean(), scores.std(), label))

In [ ]:
## 调参
X_train, X_test, y_train, y_test = train_test_split(df_6.iloc[:, 1 : ], df_6.iloc[:, 0], test_size = 0.3, shuffle = True, random_state = 10)
param = {"criterion": ["gini", "entropy"], "max_depth": np.arange(1, 20, 1), 'n_estimators': np.arange(100, 501, 50)}
rf_gs = GridSearchCV(estimator = RandomForestClassifier(), param_grid = param, cv = 5, scoring = "accuracy", n_jobs = -1, verbose = 10)
rf_gs.fit(X_train, y_train)
print(rf_gs.best_params_)
y_hat = rf_gs.best_estimator_.predict(X_test)
print(classification_report(y_test, y_hat))

In [ ]:
## OVR算法
plt.rcParams['font.sans-serif'] = ['SimHei']
X_train, X_test, y_train, y_test = train_test_split(df_6.iloc[:, 1 : ], df_6.iloc[:, 0], test_size = 0.3, shuffle = True, random_state = 10)
m1 = RandomForestClassifier(n_estimators = 300 , max_depth = 5, random_state = 10)
m2 = DecisionTreeClassifier(random_state = 10, max_depth = 5)
m3 = GradientBoostingClassifier(random_state = 10, n_estimators = 300, max_depth = 5)
m4 = LGBMClassifier(random_state = 10, n_estimators = 300, max_depth = 5)
m5 = XGBClassifier(random_state = 10, n_estimators = 300)
models =[m1, m2, m3, m4, m5]
scores = []
for model in models:
    clf_multilabel = OneVsRestClassifier(model)
    clf_multilabel.fit(X_train, y_train)
    y_pred = clf_multilabel.predict(X_test)
    score = accuracy_score(y_test, y_pred)
    print('分类报告为:%s'%(model))
    print(classification_report(y_test, y_pred))
    print('%s五折交叉验证的分数为:'%(model))
    print(cross_val_score(clf_multilabel, X_train, y_train, cv = 5))
    scores.append(score)
model_names = ('Random Forest',
               'Decision Tree',
               'GBDT',
               'LightGBM',
               'Xgboost')
x_pos = np.arange(len(model_names))
fig, ax = plt.subplots(figsize = (12, 8), dpi = 512)
ax.grid(True)
ax.set_title('One Vs Rest Classifier', font = {'family' : 'STSong', 'size' : 14})
ax.set_xticks(x_pos)
ax.set_xticklabels(model_names, font = {'family' : 'STSong', 'size' : 14})
ax.set_ylabel('model test scores', font = {'family' : 'STSong', 'size' : 14})
ax.set_ylim([min(scores) * .9, max(scores) * 1.05])
colors = ['r'] + ['b'] * (len(scores))
ax.bar(x_pos, scores, alpha = 0.5, color = colors)
plt.tight_layout()
plt.show()

In [ ]:
param = {"estimator__max_depth": np.arange(1,20,1), 'estimator__n_estimators': np.arange(100, 501, 50)}
model = OneVsRestClassifier(RandomForestClassifier(random_state = 10))
clf = GridSearchCV(model, param_grid = param, scoring = 'accuracy', n_jobs = -1, cv = 5, verbose = 2)
clf.fit(X_train, y_train)
print(clf.best_params_)
y_hat = clf.best_estimator_.predict(X_test)
print(classification_report(y_test, y_hat))

In [ ]:
## OVO算法
plt.rcParams['font.sans-serif'] = ['SimHei']
X_train, X_test, y_train, y_test = train_test_split(df_6.iloc[:, 1 : ], df_6.iloc[:, 0], test_size = 0.3, shuffle = True, random_state = 10)
m1 = RandomForestClassifier(n_estimators = 300 , max_depth = 5, random_state = 10)
m2 = DecisionTreeClassifier(random_state = 10, max_depth = 5)
m3 = GradientBoostingClassifier(random_state = 10, n_estimators = 300, max_depth = 5)
m4 = LGBMClassifier(random_state = 10, n_estimators = 300, max_depth = 5)
m5 = XGBClassifier(random_state = 10, n_estimators = 300)
models =[m1, m2, m3, m4, m5]
scores = []
for model in models:
    clf_multilabel = OneVsOneClassifier(model)
    clf_multilabel.fit(X_train, y_train)
    y_pred = clf_multilabel.predict(X_test)
    score = accuracy_score(y_test, y_pred)
    print('分类报告为:%s'%(model))
    print(classification_report(y_test, y_pred))
    print('%s五折交叉验证的分数为:'%(model))
    print(cross_val_score(clf_multilabel, X_train, y_train, cv = 5))
    scores.append(score)
model_names = ('Random Forest',
               'Decision Tree',
               'GBDT',
               'LightGBM',
               'Xgboost')
x_pos = np.arange(len(model_names))
fig, ax = plt.subplots(figsize = (12, 8), dpi = 512)
ax.grid(True)
ax.set_title('One Vs One Classifier', font = {'family' : 'STSong', 'size' : 14})
ax.set_xticks(x_pos)
ax.set_xticklabels(model_names, font = {'family' : 'STSong', 'size' : 14})
ax.set_ylabel('model test scores', font = {'family' : 'STSong', 'size' : 14})
ax.set_ylim([min(scores) * .9, max(scores) * 1.05])
colors = ['r'] + ['b'] * (len(scores))
ax.bar(x_pos, scores, alpha = 0.5, color = colors)
plt.tight_layout()
plt.show()

In [ ]:
param = {"estimator__max_depth": np.arange(1,20,1), 'estimator__n_estimators': np.arange(100, 501, 50)}
model = OneVsOneClassifier(RandomForestClassifier(random_state = 10))
clf = GridSearchCV(model, param_grid = param, scoring = 'accuracy', n_jobs = -1, cv = 5, verbose = 2)
clf.fit(X_train, y_train)
print(clf.best_params_)
y_hat = clf.best_estimator_.predict(X_test)
print(classification_report(y_test, y_hat))

In [ ]:
# 开始采用过采样
cols = ['语音通话整体满意度', '是否遇到过网络问题', 'GPRS总流量（KB）', '当月ARPU', '前3月MOU', '当月MOU', '语音通话-时长（分钟）', 'mos质差次数', \
    '终端品牌', '脱网次数', '通话中有杂音、听不清、断断续续', 'GPRS-国内漫游-流量（KB）', '未接通掉话次数', '有信号无法拨通', '客户星级标识', \
        '办公室', '前第3个月欠费金额', '通话过程中突然中断']
df_7 = pd.DataFrame()
for i in cols:
    df_7[i] = data_3[i]


df_7 = pd.get_dummies(df_7)

In [ ]:
from imblearn.over_sampling import SMOTE
x = df_7.iloc[:, 1 :]
y = df_7.iloc[:, 0 ]
smote_model = SMOTE()
x_smote, y_smote = smote_model.fit_resample(x, y)
smote_df = pd.concat([y_smote, x_smote], axis=1)
df = smote_df.copy()

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(df.iloc[:, 1 : ], df.iloc[:, 0], test_size = 0.3, shuffle = True, random_state = 10)
clf1 = XGBClassifier(random_state=10)
clf2 = RandomForestClassifier(random_state = 10)
clf3 = LGBMClassifier(random_state = 10)
clf4 = GradientBoostingClassifier(random_state = 10)
eclf = VotingClassifier(estimators=[('xgb', clf1), ('rf', clf2), ('lgb', clf3), ('gbdt', clf4)], voting = 'hard')
y_test = y_test - 1
for clf, label in zip([clf1, clf2, clf3, clf4, eclf], ['XGBBoosting', 'Random Forest', 'LightGBM', 'GBDT', 'Ensemble']):
    scores = cross_val_score(clf, X_train, y_train - 1, cv = 5, scoring='accuracy')
    print("Accuracy: %0.2f (+/- %0.2f) [%s]" % (scores.mean(), scores.std(), label))

In [ ]:
sclf = StackingClassifier(classifiers = [clf1, clf3], meta_classifier = clf2)
for clf, label in zip([clf1, clf3, sclf], ['XGboost', 'LightGBM', 'Stacking Model']):
    scores = cross_val_score(clf, X_train, y_train - 1, cv = 5, scoring = 'accuracy')
    print("Accuracy: %0.2f (+/- %0.2f) [%s]" % (scores.mean(), scores.std(), label))


sclf = StackingClassifier(classifiers = [clf2, clf3], meta_classifier = clf1)
for clf, label in zip([clf2, clf3, sclf], ['Random Forest', 'LightGBM', 'Stacking Model']):
    scores = cross_val_score(clf, X_train, y_train - 1, cv = 5, scoring = 'accuracy')
    print("Accuracy: %0.2f (+/- %0.2f) [%s]" % (scores.mean(), scores.std(), label))


sclf = StackingClassifier(classifiers = [clf1, clf2], meta_classifier = clf3)
for clf, label in zip([clf1, clf2, sclf], ['XGboost', 'Random Forest', 'Stacking Model']):
    scores = cross_val_score(clf, X_train, y_train - 1, cv = 5, scoring = 'accuracy')
    print("Accuracy: %0.2f (+/- %0.2f) [%s]" % (scores.mean(), scores.std(), label))

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(df.iloc[:, 1 : ], df.iloc[:, 0], test_size = 0.3, shuffle = True, random_state = 10)
model = XGBClassifier(max_depth = 5, n_estimators = 300, random_state = 10)
model.fit(X_train, y_train - 1)
y_pred = model.predict(X_test)
print(classification_report(y_test - 1, y_pred))

In [ ]:
score = cross_val_score(model, X_train, y_train - 1, cv = 5)

In [ ]:
plt.rcParams['font.sans-serif'] = ['SimHei']
X_train, X_test, y_train, y_test = train_test_split(df.iloc[:, 1 : ], df.iloc[:, 0], test_size = 0.3, shuffle = True, random_state = 10)
m1 = RandomForestClassifier(random_state = 10)
m2 = DecisionTreeClassifier(random_state = 10)
m3 = GradientBoostingClassifier(random_state = 10)
m4 = LGBMClassifier(random_state = 10)
m5 = XGBClassifier(random_state = 10)
models =[m1, m2, m3, m4, m5]
scores = []
for model in models:
    clf_multilabel = OneVsRestClassifier(model)
    clf_multilabel.fit(X_train, y_train)
    y_pred = clf_multilabel.predict(X_test)
    score = accuracy_score(y_test, y_pred)
    print('分类报告为:%s'%(model))
    print(classification_report(y_test, y_pred))
    print('%s五折交叉验证的分数为:'%(model))
    print(cross_val_score(clf_multilabel, X_train, y_train, cv = 5))
    scores.append(score)
model_names = ('Random Forest',
               'Decision Tree',
               'GBDT',
               'LightGBM',
               'Xgboost')
x_pos = np.arange(len(model_names))
fig, ax = plt.subplots(figsize = (12, 8), dpi = 512)
ax.grid(True)
ax.set_title('One Vs Rest Classifier', font = {'family' : 'STSong', 'size' : 14})
ax.set_xticks(x_pos)
ax.set_xticklabels(model_names, font = {'family' : 'STSong', 'size' : 14})
ax.set_ylabel('model test scores', font = {'family' : 'STSong', 'size' : 14})
ax.set_ylim([min(scores) * .9, max(scores) * 1.05])
colors = ['b'] * (len(scores) - 1) + ['r']
ax.bar(x_pos, scores, alpha = 0.5, color = colors)
plt.tight_layout()
plt.show()

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(df.iloc[:, 1 : ], df.iloc[:, 0], test_size = 0.3, shuffle = True, random_state = 10)
model = OneVsRestClassifier(XGBClassifier(max_depth = 5, n_estimators = 300, random_state = 10))
model.fit(X_train, y_train - 1)
y_pred = model.predict(X_test)
print(cross_val_score(model, X_train, y_train - 1, cv = 5))
print(classification_report(y_test - 1, y_pred))

In [ ]:
plt.rcParams['font.sans-serif'] = ['SimHei']
X_train, X_test, y_train, y_test = train_test_split(df.iloc[:, 1 : ], df.iloc[:, 0], test_size = 0.3, shuffle = True, random_state = 10)
m1 = RandomForestClassifier(random_state = 10)
m2 = DecisionTreeClassifier(random_state = 10)
m3 = GradientBoostingClassifier(random_state = 10)
m4 = LGBMClassifier(random_state = 10)
m5 = XGBClassifier(random_state = 10)
models =[m1, m2, m3, m4, m5]
scores = []
for model in models:
    clf_multilabel = OneVsOneClassifier(model)
    clf_multilabel.fit(X_train, y_train)
    y_pred = clf_multilabel.predict(X_test)
    score = accuracy_score(y_test, y_pred)
    print('分类报告为:%s'%(model))
    print(classification_report(y_test, y_pred))
    print('%s五折交叉验证的分数为:'%(model))
    print(cross_val_score(clf_multilabel, X_train, y_train, cv = 5))
    scores.append(score)
model_names = ('Random Forest',
               'Decision Tree',
               'GBDT',
               'LightGBM',
               'Xgboost')
x_pos = np.arange(len(model_names))
fig, ax = plt.subplots(figsize = (12, 8), dpi = 512)
ax.grid(True)
ax.set_title('One Vs One Classifier', font = {'family' : 'STSong', 'size' : 14})
ax.set_xticks(x_pos)
ax.set_xticklabels(model_names, font = {'family' : 'STSong', 'size' : 14})
ax.set_ylabel('model test scores', font = {'family' : 'STSong', 'size' : 14})
ax.set_ylim([min(scores) * .9, max(scores) * 1.05])
colors = ['b'] * (len(scores) - 1) + ['r']
ax.bar(x_pos, scores, alpha = 0.5, color = colors)
plt.tight_layout()
plt.show()

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(df.iloc[:, 1 : ], df.iloc[:, 0], test_size = 0.3, shuffle = True, random_state = 10)
model = OneVsOneClassifier(XGBClassifier(max_depth = 5, n_estimators = 300, random_state = 10))
model.fit(X_train, y_train - 1)
y_pred = model.predict(X_test)
print(cross_val_score(model, X_train, y_train - 1, cv = 5))
print(classification_report(y_test - 1, y_pred))

In [ ]:
data = pd.read_excel('./data/附件3语音业务用户满意度预测数据.xlsx')
data = data.drop('用户id', axis = 1)

In [ ]:
cols = ['是否遇到过网络问题', 'GPRS总流量（KB）', '当月ARPU', '前3月MOU', '当月MOU', '语音通话-时长（分钟）', 'mos质差次数', \
    '脱网次数', '通话中有杂音、听不清、断断续续', 'GPRS-国内漫游-流量（KB）', '未接通掉话次数', '有信号无法拨通', '客户星级标识', \
        '办公室', '通话过程中突然中断']
df_8 = pd.DataFrame()
for i in cols:
    df_8[i] = data[i]

In [ ]:
df_8['是否遇到过网络问题'].replace(1, '是', inplace = True)
df_8['是否遇到过网络问题'].replace(2, '否', inplace = True)
df_8['通话中有杂音、听不清、断断续续'].replace(-1, '没有', inplace = True)
df_8['通话中有杂音、听不清、断断续续'].replace(4, '有', inplace = True)
df_8['有信号无法拨通'].replace(-1, '没有', inplace = True)
df_8['有信号无法拨通'].replace(2, '有', inplace = True)
df_8['办公室'].replace(-1, '不在', inplace = True)
df_8['办公室'].replace(2, '在', inplace = True)
df_8['通话过程中突然中断'].replace(-1, '没有', inplace = True)
df_8['通话过程中突然中断'].replace(3, '有', inplace = True)
df_8 = pd.get_dummies(df_8)

In [ ]:
cols = ['语音通话整体满意度' ,'是否遇到过网络问题', 'GPRS总流量（KB）', '当月ARPU', '前3月MOU', '当月MOU', '语音通话-时长（分钟）', 'mos质差次数', \
    '脱网次数', '通话中有杂音、听不清、断断续续', 'GPRS-国内漫游-流量（KB）', '未接通掉话次数', '有信号无法拨通', '客户星级标识', \
        '办公室', '通话过程中突然中断']
df_9 = pd.DataFrame()
for i in cols:
    df_9[i] = data_3[i]

In [ ]:
from imblearn.over_sampling import SMOTE
df_9 = pd.get_dummies(df_9)
x = df_9.iloc[:, 1 :]
y = df_9.iloc[:, 0 ]
smote_model = SMOTE()
x_smote, y_smote = smote_model.fit_resample(x, y)
smote_df = pd.concat([y_smote, x_smote], axis=1)
df = smote_df.copy()

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(df.iloc[:, 1 : ], df.iloc[:, 0], test_size = 0.3, shuffle = True, random_state = 10)
model = OneVsRestClassifier(XGBClassifier(n_estimators = 300, max_depth = 5, random_state = 10))
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
print(classification_report(y_test, y_pred))

In [ ]:
y_pred_1 = model.predict(df_8)

In [ ]:
pred_data = pd.read_excel('./result.xlsx')
pred_data['语音通话整体满意度'] = y_pred_1

In [ ]:
cols = ['网络覆盖与信号强度', '是否遇到过网络问题', 'GPRS总流量（KB）', '当月ARPU', '前3月MOU', '当月MOU', '语音通话-时长（分钟）', 'mos质差次数', \
    '终端品牌', '脱网次数', '通话中有杂音、听不清、断断续续', 'GPRS-国内漫游-流量（KB）', '未接通掉话次数', '有信号无法拨通', '客户星级标识', \
        '办公室', '前第3个月欠费金额', '通话过程中突然中断']
df_10 = pd.DataFrame()
for i in cols:
    df_10[i] = data_3[i]
df_10 = pd.get_dummies(df_10)

In [ ]:
from imblearn.over_sampling import SMOTE
x = df_10.iloc[:, 1 :]
y = df_10.iloc[:, 0 ]
smote_model = SMOTE()
x_smote, y_smote = smote_model.fit_resample(x, y)
smote_df = pd.concat([y_smote, x_smote], axis = 1)
df = smote_df.copy()

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(df.iloc[:, 1 : ], df.iloc[:, 0], test_size = 0.3, shuffle = True, random_state = 10)
clf1 = XGBClassifier(random_state=10)
clf2 = RandomForestClassifier(random_state = 10)
clf3 = LGBMClassifier(random_state = 10)
clf4 = GradientBoostingClassifier(random_state = 10)
eclf = VotingClassifier(estimators=[('xgb', clf1), ('rf', clf2), ('lgb', clf3), ('gbdt', clf4)], voting = 'hard')
y_test = y_test - 1
for clf, label in zip([clf1, clf2, clf3, clf4, eclf], ['XGBBoosting', 'Random Forest', 'LightGBM', 'GBDT', 'Ensemble']):
    scores = cross_val_score(clf, X_train, y_train - 1, cv = 5, scoring='accuracy')
    print("Accuracy: %0.2f (+/- %0.2f) [%s]" % (scores.mean(), scores.std(), label))

In [ ]:
sclf = StackingClassifier(classifiers = [clf1, clf3], meta_classifier = clf2)
for clf, label in zip([clf1, clf3, sclf], ['XGboost', 'LightGBM', 'Stacking Model']):
    scores = cross_val_score(clf, X_train, y_train - 1, cv = 5, scoring = 'accuracy')
    print("Accuracy: %0.2f (+/- %0.2f) [%s]" % (scores.mean(), scores.std(), label))


sclf = StackingClassifier(classifiers = [clf2, clf3], meta_classifier = clf1)
for clf, label in zip([clf2, clf3, sclf], ['Random Forest', 'LightGBM', 'Stacking Model']):
    scores = cross_val_score(clf, X_train, y_train - 1, cv = 5, scoring = 'accuracy')
    print("Accuracy: %0.2f (+/- %0.2f) [%s]" % (scores.mean(), scores.std(), label))


sclf = StackingClassifier(classifiers = [clf1, clf2], meta_classifier = clf3)
for clf, label in zip([clf1, clf2, sclf], ['XGboost', 'Random Forest', 'Stacking Model']):
    scores = cross_val_score(clf, X_train, y_train - 1, cv = 5, scoring = 'accuracy')
    print("Accuracy: %0.2f (+/- %0.2f) [%s]" % (scores.mean(), scores.std(), label))

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(df.iloc[:, 1 : ], df.iloc[:, 0], test_size = 0.3, shuffle = True, random_state = 10)
model = XGBClassifier(max_depth = 5, n_estimators = 300, random_state = 10)
model.fit(X_train, y_train - 1)
y_pred = model.predict(X_test)
print(classification_report(y_test - 1, y_pred))

In [ ]:
plt.rcParams['font.sans-serif'] = ['SimHei']
X_train, X_test, y_train, y_test = train_test_split(df.iloc[:, 1 : ], df.iloc[:, 0], test_size = 0.3, shuffle = True, random_state = 10)
m1 = RandomForestClassifier(random_state = 10)
m2 = DecisionTreeClassifier(random_state = 10)
m3 = GradientBoostingClassifier(random_state = 10)
m4 = LGBMClassifier(random_state = 10)
m5 = XGBClassifier(random_state = 10)
models =[m1, m2, m3, m4, m5]
scores = []
for model in models:
    clf_multilabel = OneVsRestClassifier(model)
    clf_multilabel.fit(X_train, y_train)
    y_pred = clf_multilabel.predict(X_test)
    score = accuracy_score(y_test, y_pred)
    print('分类报告为:%s'%(model))
    print(classification_report(y_test, y_pred))
    print('%s五折交叉验证的分数为:'%(model))
    print(cross_val_score(clf_multilabel, X_train, y_train, cv = 5))
    scores.append(score)
model_names = ('Random Forest',
               'Decision Tree',
               'GBDT',
               'LightGBM',
               'Xgboost')
x_pos = np.arange(len(model_names))
fig, ax = plt.subplots(figsize = (12, 8), dpi = 512)
ax.grid(True)
ax.set_title('One Vs Rest Classifier', font = {'family' : 'STSong', 'size' : 14})
ax.set_xticks(x_pos)
ax.set_xticklabels(model_names, font = {'family' : 'STSong', 'size' : 14})
ax.set_ylabel('model test scores', font = {'family' : 'STSong', 'size' : 14})
ax.set_ylim([min(scores) * .9, max(scores) * 1.05])
colors = ['b'] * (len(scores) - 1) + ['r']
ax.bar(x_pos, scores, alpha = 0.5, color = colors)
plt.tight_layout()
plt.show()

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(df.iloc[:, 1 : ], df.iloc[:, 0], test_size = 0.3, shuffle = True, random_state = 10)
model = OneVsRestClassifier(XGBClassifier(max_depth = 5, n_estimators = 300, random_state = 10))
model.fit(X_train, y_train - 1)
y_pred = model.predict(X_test)
print(cross_val_score(model, X_train, y_train - 1, cv = 5))
print(classification_report(y_test - 1, y_pred))

In [ ]:
plt.rcParams['font.sans-serif'] = ['SimHei']
X_train, X_test, y_train, y_test = train_test_split(df.iloc[:, 1 : ], df.iloc[:, 0], test_size = 0.3, shuffle = True, random_state = 10)
m1 = RandomForestClassifier(random_state = 10)
m2 = DecisionTreeClassifier(random_state = 10)
m3 = GradientBoostingClassifier(random_state = 10)
m4 = LGBMClassifier(random_state = 10)
m5 = XGBClassifier(random_state = 10)
models =[m1, m2, m3, m4, m5]
scores = []
for model in models:
    clf_multilabel = OneVsOneClassifier(model)
    clf_multilabel.fit(X_train, y_train)
    y_pred = clf_multilabel.predict(X_test)
    score = accuracy_score(y_test, y_pred)
    print('分类报告为:%s'%(model))
    print(classification_report(y_test, y_pred))
    print('%s五折交叉验证的分数为:'%(model))
    print(cross_val_score(clf_multilabel, X_train, y_train, cv = 5))
    scores.append(score)
model_names = ('Random Forest',
               'Decision Tree',
               'GBDT',
               'LightGBM',
               'Xgboost')
x_pos = np.arange(len(model_names))
fig, ax = plt.subplots(figsize = (12, 8), dpi = 512)
ax.grid(True)
ax.set_title('One Vs One Classifier', font = {'family' : 'STSong', 'size' : 14})
ax.set_xticks(x_pos)
ax.set_xticklabels(model_names, font = {'family' : 'STSong', 'size' : 14})
ax.set_ylabel('model test scores', font = {'family' : 'STSong', 'size' : 14})
ax.set_ylim([min(scores) * .9, max(scores) * 1.05])
colors = ['b'] * (len(scores) - 1) + ['r']
ax.bar(x_pos, scores, alpha = 0.5, color = colors)
plt.tight_layout()
plt.show()

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(df.iloc[:, 1 : ], df.iloc[:, 0], test_size = 0.3, shuffle = True, random_state = 10)
model = OneVsOneClassifier(XGBClassifier(max_depth = 5, n_estimators = 300, random_state = 10))
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
print(cross_val_score(model, X_train, y_train, cv = 5))
print(classification_report(y_test, y_pred))

In [ ]:
cols = ['是否遇到过网络问题', 'GPRS总流量（KB）', '当月ARPU', '前3月MOU', '当月MOU', '语音通话-时长（分钟）', 'mos质差次数', \
    '脱网次数', '通话中有杂音、听不清、断断续续', 'GPRS-国内漫游-流量（KB）', '未接通掉话次数', '有信号无法拨通', '客户星级标识', \
        '办公室', '通话过程中突然中断']
df_11 = pd.DataFrame()
for i in cols:
    df_11[i] = data[i]

In [ ]:
df_11['是否遇到过网络问题'].replace(1, '是', inplace = True)
df_11['是否遇到过网络问题'].replace(2, '否', inplace = True)
df_11['通话中有杂音、听不清、断断续续'].replace(-1, '没有', inplace = True)
df_11['通话中有杂音、听不清、断断续续'].replace(4, '有', inplace = True)
df_11['有信号无法拨通'].replace(-1, '没有', inplace = True)
df_11['有信号无法拨通'].replace(2, '有', inplace = True)
df_11['办公室'].replace(-1, '不在', inplace = True)
df_11['办公室'].replace(2, '在', inplace = True)
df_11['通话过程中突然中断'].replace(-1, '没有', inplace = True)
df_11['通话过程中突然中断'].replace(3, '有', inplace = True)
df_11 = pd.get_dummies(df_11)

In [ ]:
cols = ['网络覆盖与信号强度' ,'是否遇到过网络问题', 'GPRS总流量（KB）', '当月ARPU', '前3月MOU', '当月MOU', '语音通话-时长（分钟）', 'mos质差次数', \
    '脱网次数', '通话中有杂音、听不清、断断续续', 'GPRS-国内漫游-流量（KB）', '未接通掉话次数', '有信号无法拨通', '客户星级标识', \
        '办公室', '通话过程中突然中断']
df_12 = pd.DataFrame()
for i in cols:
    df_12[i] = data_3[i]

In [ ]:
df_12 = pd.get_dummies(df_12)
x = df_12.iloc[:, 1 :]
y = df_12.iloc[:, 0 ]
smote_model = SMOTE()
x_smote, y_smote = smote_model.fit_resample(x, y)
smote_df = pd.concat([y_smote, x_smote], axis=1)
df = smote_df.copy()

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(df.iloc[:, 1 : ], df.iloc[:, 0], test_size = 0.3, shuffle = True, random_state = 10)
model = OneVsRestClassifier(XGBClassifier(max_depth = 5, n_estimators = 300, random_state = 10))
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
print(classification_report(y_test, y_pred))

In [ ]:
y_pred_2 = model.predict(df_11)
pred_data['网络覆盖与信号强度'] = y_pred_2

In [ ]:
cols = ['语音通话清晰度', '是否遇到过网络问题', 'GPRS总流量（KB）', '当月ARPU', '前3月MOU', '当月MOU', '语音通话-时长（分钟）', 'mos质差次数', \
    '终端品牌', '脱网次数', '通话中有杂音、听不清、断断续续', 'GPRS-国内漫游-流量（KB）', '未接通掉话次数', '有信号无法拨通', '客户星级标识', \
        '办公室', '前第3个月欠费金额', '通话过程中突然中断']
df_13 = pd.DataFrame()
for i in cols:
    df_13[i] = data_3[i]
df_13 = pd.get_dummies(df_13)

In [ ]:
x = df_13.iloc[:, 1 :]
y = df_13.iloc[:, 0 ]
smote_model = SMOTE()
x_smote, y_smote = smote_model.fit_resample(x, y)
smote_df = pd.concat([y_smote, x_smote], axis=1)
df = smote_df.copy()

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(df.iloc[:, 1 : ], df.iloc[:, 0], test_size = 0.3, shuffle = True, random_state = 10)
clf1 = XGBClassifier(random_state=10)
clf2 = RandomForestClassifier(random_state = 10)
clf3 = LGBMClassifier(random_state = 10)
clf4 = GradientBoostingClassifier(random_state = 10)
eclf = VotingClassifier(estimators=[('xgb', clf1), ('rf', clf2), ('lgb', clf3), ('gbdt', clf4)], voting = 'hard')
y_test = y_test - 1
for clf, label in zip([clf1, clf2, clf3, clf4, eclf], ['XGBBoosting', 'Random Forest', 'LightGBM', 'GBDT', 'Ensemble']):
    scores = cross_val_score(clf, X_train, y_train - 1, cv = 5, scoring='accuracy')
    print("Accuracy: %0.2f (+/- %0.2f) [%s]" % (scores.mean(), scores.std(), label))

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(df.iloc[:, 1 : ], df.iloc[:, 0], test_size = 0.3, shuffle = True, random_state = 10)
clf1 = XGBClassifier(random_state=10)
clf2 = RandomForestClassifier(random_state = 10)
clf3 = LGBMClassifier(random_state = 10)
eclf = VotingClassifier(estimators=[('xgb', clf1), ('rf', clf2), ('lgb', clf3)], voting = 'hard')
y_test = y_test - 1
for clf, label in zip([clf1, clf2, clf3, clf4, eclf], ['XGBBoosting', 'Random Forest', 'LightGBM', 'GBDT', 'Ensemble']):
    scores = cross_val_score(clf, X_train, y_train - 1, cv = 5, scoring='accuracy')
    print("Accuracy: %0.2f (+/- %0.2f) [%s]" % (scores.mean(), scores.std(), label))

In [ ]:
sclf = StackingClassifier(classifiers = [clf1, clf3], meta_classifier = clf2)
for clf, label in zip([clf1, clf3, sclf], ['XGboost', 'LightGBM', 'Stacking Model']):
    scores = cross_val_score(clf, X_train, y_train - 1, cv = 5, scoring = 'accuracy')
    print("Accuracy: %0.2f (+/- %0.2f) [%s]" % (scores.mean(), scores.std(), label))

In [ ]:
sclf = StackingClassifier(classifiers = [clf2, clf3], meta_classifier = clf1)
for clf, label in zip([clf2, clf3, sclf], ['Random Forest', 'LightGBM', 'Stacking Model']):
    scores = cross_val_score(clf, X_train, y_train - 1, cv = 5, scoring = 'accuracy')
    print("Accuracy: %0.2f (+/- %0.2f) [%s]" % (scores.mean(), scores.std(), label))


sclf = StackingClassifier(classifiers = [clf1, clf2], meta_classifier = clf3)
for clf, label in zip([clf1, clf2, sclf], ['XGboost', 'Random Forest', 'Stacking Model']):
    scores = cross_val_score(clf, X_train, y_train - 1, cv = 5, scoring = 'accuracy')
    print("Accuracy: %0.2f (+/- %0.2f) [%s]" % (scores.mean(), scores.std(), label))

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(df.iloc[:, 1 : ], df.iloc[:, 0], test_size = 0.3, shuffle = True, random_state = 10)
model = XGBClassifier(max_depth = 5, n_estimators = 300, random_state = 10)
model.fit(X_train, y_train - 1)
y_pred = model.predict(X_test)
print(classification_report(y_test - 1, y_pred))

In [ ]:
score = cross_val_score(model, X_train, y_train - 1, cv = 5)

In [ ]:
plt.rcParams['font.sans-serif'] = ['SimHei']
X_train, X_test, y_train, y_test = train_test_split(df.iloc[:, 1 : ], df.iloc[:, 0], test_size = 0.3, shuffle = True, random_state = 10)
m1 = RandomForestClassifier(random_state = 10)
m2 = DecisionTreeClassifier(random_state = 10)
m3 = GradientBoostingClassifier(random_state = 10)
m4 = LGBMClassifier(random_state = 10)
m5 = XGBClassifier(random_state = 10)
models =[m1, m2, m3, m4, m5]
scores = []
for model in models:
    clf_multilabel = OneVsRestClassifier(model)
    clf_multilabel.fit(X_train, y_train)
    y_pred = clf_multilabel.predict(X_test)
    score = accuracy_score(y_test, y_pred)
    print('分类报告为:%s'%(model))
    print(classification_report(y_test, y_pred))
    print('%s五折交叉验证的分数为:'%(model))
    print(cross_val_score(clf_multilabel, X_train, y_train, cv = 5))
    scores.append(score)
model_names = ('Random Forest',
               'Decision Tree',
               'GBDT',
               'LightGBM',
               'Xgboost')
x_pos = np.arange(len(model_names))
fig, ax = plt.subplots(figsize = (12, 8), dpi = 512)
ax.grid(True)
ax.set_title('One Vs Rest Classifier', font = {'family' : 'STSong', 'size' : 14})
ax.set_xticks(x_pos)
ax.set_xticklabels(model_names, font = {'family' : 'STSong', 'size' : 14})
ax.set_ylabel('model test scores', font = {'family' : 'STSong', 'size' : 14})
ax.set_ylim([min(scores) * .9, max(scores) * 1.05])
colors = ['b'] * (len(scores) - 1) + ['r']
ax.bar(x_pos, scores, alpha = 0.5, color = colors)
plt.tight_layout()
plt.show()

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(df.iloc[:, 1 : ], df.iloc[:, 0], test_size = 0.3, shuffle = True, random_state = 10)
model = OneVsRestClassifier(XGBClassifier(max_depth = 5, n_estimators = 300, random_state = 10))
model.fit(X_train, y_train - 1)
y_pred = model.predict(X_test)
print(cross_val_score(model, X_train, y_train - 1, cv = 5))
print(classification_report(y_test - 1, y_pred))


plt.rcParams['font.sans-serif'] = ['SimHei']
X_train, X_test, y_train, y_test = train_test_split(df.iloc[:, 1 : ], df.iloc[:, 0], test_size = 0.3, shuffle = True, random_state = 10)
m1 = RandomForestClassifier(random_state = 10)
m2 = DecisionTreeClassifier(random_state = 10)
m3 = GradientBoostingClassifier(random_state = 10)
m4 = LGBMClassifier(random_state = 10)
m5 = XGBClassifier(random_state = 10)
models =[m1, m2, m3, m4, m5]
scores = []
for model in models:
    clf_multilabel = OneVsOneClassifier(model)
    clf_multilabel.fit(X_train, y_train)
    y_pred = clf_multilabel.predict(X_test)
    score = accuracy_score(y_test, y_pred)
    print('分类报告为:%s'%(model))
    print(classification_report(y_test, y_pred))
    print('%s五折交叉验证的分数为:'%(model))
    print(cross_val_score(clf_multilabel, X_train, y_train, cv = 5))
    scores.append(score)
model_names = ('Random Forest',
               'Decision Tree',
               'GBDT',
               'LightGBM',
               'Xgboost')
x_pos = np.arange(len(model_names))
fig, ax = plt.subplots(figsize = (12, 8), dpi = 512)
ax.grid(True)
ax.set_title('One Vs One Classifier', font = {'family' : 'STSong', 'size' : 14})
ax.set_xticks(x_pos)
ax.set_xticklabels(model_names, font = {'family' : 'STSong', 'size' : 14})
ax.set_ylabel('model test scores', font = {'family' : 'STSong', 'size' : 14})
ax.set_ylim([min(scores) * .9, max(scores) * 1.05])
colors = ['b'] * (len(scores) - 1) + ['r']
ax.bar(x_pos, scores, alpha = 0.5, color = colors)
plt.tight_layout()
plt.show()

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(df.iloc[:, 1 : ], df.iloc[:, 0], test_size = 0.3, shuffle = True, random_state = 10)
model = OneVsOneClassifier(XGBClassifier(max_depth = 5, n_estimators = 300, random_state = 10))
model.fit(X_train, y_train - 1)
y_pred = model.predict(X_test)
print(cross_val_score(model, X_train, y_train - 1, cv = 5))
print(classification_report(y_test - 1, y_pred))

In [ ]:
cols = ['是否遇到过网络问题', 'GPRS总流量（KB）', '当月ARPU', '前3月MOU', '当月MOU', '语音通话-时长（分钟）', 'mos质差次数', \
    '脱网次数', '通话中有杂音、听不清、断断续续', 'GPRS-国内漫游-流量（KB）', '未接通掉话次数', '有信号无法拨通', '客户星级标识', \
        '办公室', '通话过程中突然中断']
df_14 = pd.DataFrame()
for i in cols:
    df_14[i] = data[i]

In [ ]:
df_14['是否遇到过网络问题'].replace(1, '是', inplace = True)
df_14['是否遇到过网络问题'].replace(2, '否', inplace = True)
df_14['通话中有杂音、听不清、断断续续'].replace(-1, '没有', inplace = True)
df_14['通话中有杂音、听不清、断断续续'].replace(4, '有', inplace = True)
df_14['有信号无法拨通'].replace(-1, '没有', inplace = True)
df_14['有信号无法拨通'].replace(2, '有', inplace = True)
df_14['办公室'].replace(-1, '不在', inplace = True)
df_14['办公室'].replace(2, '在', inplace = True)
df_14['通话过程中突然中断'].replace(-1, '没有', inplace = True)
df_14['通话过程中突然中断'].replace(3, '有', inplace = True)
df_14 = pd.get_dummies(df_14)

In [ ]:
cols = ['语音通话清晰度' ,'是否遇到过网络问题', 'GPRS总流量（KB）', '当月ARPU', '前3月MOU', '当月MOU', '语音通话-时长（分钟）', 'mos质差次数', \
    '脱网次数', '通话中有杂音、听不清、断断续续', 'GPRS-国内漫游-流量（KB）', '未接通掉话次数', '有信号无法拨通', '客户星级标识', \
        '办公室', '通话过程中突然中断']
df_15 = pd.DataFrame()
for i in cols:
    df_15[i] = data_3[i]

In [ ]:
df_15 = pd.get_dummies(df_15)
x = df_15.iloc[:, 1 :]
y = df_15.iloc[:, 0 ]
smote_model = SMOTE()
x_smote, y_smote = smote_model.fit_resample(x, y)
smote_df = pd.concat([y_smote, x_smote], axis=1)
df = smote_df.copy()

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(df.iloc[:, 1 : ], df.iloc[:, 0], test_size = 0.3, shuffle = True, random_state = 10)
model = OneVsRestClassifier(XGBClassifier(max_depth = 5, n_estimators = 300, random_state = 10))
model.fit(X_train, y_train)
print(classification_report(y_test, model.predict(X_test)))

In [ ]:
y_pred_3 = model.predict(df_14)
pred_data['语音通话清晰度'] = y_pred_3

In [ ]:
cols = ['语音通话稳定性', '是否遇到过网络问题', 'GPRS总流量（KB）', '当月ARPU', '前3月MOU', '当月MOU', '语音通话-时长（分钟）', 'mos质差次数', \
    '终端品牌', '脱网次数', '通话中有杂音、听不清、断断续续', 'GPRS-国内漫游-流量（KB）', '未接通掉话次数', '有信号无法拨通', '客户星级标识', \
        '办公室', '前第3个月欠费金额', '通话过程中突然中断']
df_16 = pd.DataFrame()
for i in cols:
    df_16[i] = data_3[i]
df_16 = pd.get_dummies(df_16)

In [ ]:
x = df_16.iloc[:, 1 :]
y = df_16.iloc[:, 0 ]
smote_model = SMOTE()
x_smote, y_smote = smote_model.fit_resample(x, y)
smote_df = pd.concat([y_smote, x_smote], axis=1)
df = smote_df.copy()

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(df.iloc[:, 1 : ], df.iloc[:, 0], test_size = 0.3, shuffle = True, random_state = 10)
clf1 = XGBClassifier(random_state=10)
clf2 = RandomForestClassifier(random_state = 10)
clf3 = LGBMClassifier(random_state = 10)
eclf = VotingClassifier(estimators=[('xgb', clf1), ('rf', clf2), ('lgb', clf3)], voting = 'hard')
y_test = y_test - 1
for clf, label in zip([clf1, clf2, clf3, clf4, eclf], ['XGBBoosting', 'Random Forest', 'LightGBM', 'Ensemble']):
    scores = cross_val_score(clf, X_train, y_train - 1, cv = 5, scoring='accuracy')
    print("Accuracy: %0.2f (+/- %0.2f) [%s]" % (scores.mean(), scores.std(), label))

In [ ]:
sclf = StackingClassifier(classifiers = [clf1, clf3], meta_classifier = clf2)
for clf, label in zip([clf1, clf3, sclf], ['XGboost', 'LightGBM', 'Stacking Model']):
    scores = cross_val_score(clf, X_train, y_train - 1, cv = 5, scoring = 'accuracy')
    print("Accuracy: %0.2f (+/- %0.2f) [%s]" % (scores.mean(), scores.std(), label))

In [ ]:
sclf = StackingClassifier(classifiers = [clf2, clf3], meta_classifier = clf1)
for clf, label in zip([clf2, clf3, sclf], ['Random Forest', 'LightGBM', 'Stacking Model']):
    scores = cross_val_score(clf, X_train, y_train - 1, cv = 5, scoring = 'accuracy')
    print("Accuracy: %0.2f (+/- %0.2f) [%s]" % (scores.mean(), scores.std(), label))

In [ ]:
sclf = StackingClassifier(classifiers = [clf1, clf2], meta_classifier = clf3)
for clf, label in zip([clf1, clf2, sclf], ['XGboost', 'Random Forest', 'Stacking Model']):
    scores = cross_val_score(clf, X_train, y_train - 1, cv = 5, scoring = 'accuracy')
    print("Accuracy: %0.2f (+/- %0.2f) [%s]" % (scores.mean(), scores.std(), label))

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(df.iloc[:, 1 : ], df.iloc[:, 0], test_size = 0.3, shuffle = True, random_state = 10)
model = XGBClassifier(max_depth = 5, n_estimators = 300, random_state = 10)
model.fit(X_train, y_train - 1)
y_pred = model.predict(X_test)
print(classification_report(y_test - 1, y_pred))

In [ ]:
score = cross_val_score(model, X_train, y_train - 1, cv = 5)

In [ ]:
plt.rcParams['font.sans-serif'] = ['SimHei']
X_train, X_test, y_train, y_test = train_test_split(df.iloc[:, 1 : ], df.iloc[:, 0], test_size = 0.3, shuffle = True, random_state = 10)
m1 = RandomForestClassifier(random_state = 10)
m2 = DecisionTreeClassifier(random_state = 10)
m3 = GradientBoostingClassifier(random_state = 10)
m4 = LGBMClassifier(random_state = 10)
m5 = XGBClassifier(random_state = 10)
models =[m1, m2, m3, m4, m5]
scores = []
for model in models:
    clf_multilabel = OneVsRestClassifier(model)
    clf_multilabel.fit(X_train, y_train)
    y_pred = clf_multilabel.predict(X_test)
    score = accuracy_score(y_test, y_pred)
    print('分类报告为:%s'%(model))
    print(classification_report(y_test, y_pred))
    print('%s五折交叉验证的分数为:'%(model))
    print(cross_val_score(clf_multilabel, X_train, y_train, cv = 5))
    scores.append(score)
model_names = ('Random Forest',
               'Decision Tree',
               'GBDT',
               'LightGBM',
               'Xgboost')
x_pos = np.arange(len(model_names))
fig, ax = plt.subplots(figsize = (12, 8), dpi = 512)
ax.grid(True)
ax.set_title('One Vs Rest Classifier', font = {'family' : 'STSong', 'size' : 14})
ax.set_xticks(x_pos)
ax.set_xticklabels(model_names, font = {'family' : 'STSong', 'size' : 14})
ax.set_ylabel('model test scores', font = {'family' : 'STSong', 'size' : 14})
ax.set_ylim([min(scores) * .9, max(scores) * 1.05])
colors = ['b'] * (len(scores) - 1) + ['r']
ax.bar(x_pos, scores, alpha = 0.5, color = colors)
plt.tight_layout()
plt.show()

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(df.iloc[:, 1 : ], df.iloc[:, 0], test_size = 0.3, shuffle = True, random_state = 10)
model = OneVsRestClassifier(XGBClassifier(max_depth = 5, n_estimators = 300, random_state = 10))
model.fit(X_train, y_train - 1)
y_pred = model.predict(X_test)
print(cross_val_score(model, X_train, y_train - 1, cv = 5))
print(classification_report(y_test - 1, y_pred))

In [ ]:
plt.rcParams['font.sans-serif'] = ['SimHei']
X_train, X_test, y_train, y_test = train_test_split(df.iloc[:, 1 : ], df.iloc[:, 0], test_size = 0.3, shuffle = True, random_state = 10)
m1 = RandomForestClassifier(random_state = 10)
m2 = DecisionTreeClassifier(random_state = 10)
m3 = GradientBoostingClassifier(random_state = 10)
m4 = LGBMClassifier(random_state = 10)
m5 = XGBClassifier(random_state = 10)
models =[m1, m2, m3, m4, m5]
scores = []
for model in models:
    clf_multilabel = OneVsOneClassifier(model)
    clf_multilabel.fit(X_train, y_train)
    y_pred = clf_multilabel.predict(X_test)
    score = accuracy_score(y_test, y_pred)
    print('分类报告为:%s'%(model))
    print(classification_report(y_test, y_pred))
    print('%s五折交叉验证的分数为:'%(model))
    print(cross_val_score(clf_multilabel, X_train, y_train, cv = 5))
    scores.append(score)
model_names = ('Random Forest',
               'Decision Tree',
               'GBDT',
               'LightGBM',
               'Xgboost')
x_pos = np.arange(len(model_names))
fig, ax = plt.subplots(figsize = (12, 8), dpi = 512)
ax.grid(True)
ax.set_title('One Vs One Classifier', font = {'family' : 'STSong', 'size' : 14})
ax.set_xticks(x_pos)
ax.set_xticklabels(model_names, font = {'family' : 'STSong', 'size' : 14})
ax.set_ylabel('model test scores', font = {'family' : 'STSong', 'size' : 14})
ax.set_ylim([min(scores) * .9, max(scores) * 1.05])
colors = ['b'] * (len(scores) - 1) + ['r']
ax.bar(x_pos, scores, alpha = 0.5, color = colors)
plt.tight_layout()
plt.show()

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(df.iloc[:, 1 : ], df.iloc[:, 0], test_size = 0.3, shuffle = True, random_state = 10)
model = OneVsOneClassifier(XGBClassifier(max_depth = 5, n_estimators = 300, random_state = 10))
model.fit(X_train, y_train - 1)
y_pred = model.predict(X_test)
print(cross_val_score(model, X_train, y_train - 1, cv = 5))
print(classification_report(y_test - 1, y_pred))

In [ ]:
cols = ['是否遇到过网络问题', 'GPRS总流量（KB）', '当月ARPU', '前3月MOU', '当月MOU', '语音通话-时长（分钟）', 'mos质差次数', \
    '脱网次数', '通话中有杂音、听不清、断断续续', 'GPRS-国内漫游-流量（KB）', '未接通掉话次数', '有信号无法拨通', '客户星级标识', \
        '办公室', '通话过程中突然中断']
df_17 = pd.DataFrame()
for i in cols:
    df_17[i] = data[i]

In [ ]:
df_17['是否遇到过网络问题'].replace(1, '是', inplace = True)
df_17['是否遇到过网络问题'].replace(2, '否', inplace = True)
df_17['通话中有杂音、听不清、断断续续'].replace(-1, '没有', inplace = True)
df_17['通话中有杂音、听不清、断断续续'].replace(4, '有', inplace = True)
df_17['有信号无法拨通'].replace(-1, '没有', inplace = True)
df_17['有信号无法拨通'].replace(2, '有', inplace = True)
df_17['办公室'].replace(-1, '不在', inplace = True)
df_17['办公室'].replace(2, '在', inplace = True)
df_17['通话过程中突然中断'].replace(-1, '没有', inplace = True)
df_17['通话过程中突然中断'].replace(3, '有', inplace = True)
df_17 = pd.get_dummies(df_17)

In [ ]:
cols = ['语音通话稳定性' ,'是否遇到过网络问题', 'GPRS总流量（KB）', '当月ARPU', '前3月MOU', '当月MOU', '语音通话-时长（分钟）', 'mos质差次数', \
    '脱网次数', '通话中有杂音、听不清、断断续续', 'GPRS-国内漫游-流量（KB）', '未接通掉话次数', '有信号无法拨通', '客户星级标识', \
        '办公室', '通话过程中突然中断']
df_18 = pd.DataFrame()
for i in cols:
    df_18[i] = data_3[i]

In [ ]:
df_18 = pd.get_dummies(df_18)
x = df_18.iloc[:, 1 :]
y = df_18.iloc[:, 0 ]
smote_model = SMOTE()
x_smote, y_smote = smote_model.fit_resample(x, y)
smote_df = pd.concat([y_smote, x_smote], axis=1)
df = smote_df.copy()


In [ ]:
X_train, X_test, y_train, y_test = train_test_split(df.iloc[:, 1 : ], df.iloc[:, 0], test_size = 0.3, shuffle = True, random_state = 10)
model = OneVsRestClassifier(XGBClassifier(max_depth = 5, n_estimators = 300, random_state = 10))
model.fit(X_train, y_train)
print(classification_report(y_test, model.predict(X_test)))

y_pred_4 = model.predict(df_17)
pred_data['语音通话稳定性'] = y_pred_4

pred_data.to_excel('./data/predict1.xlsx')